<a href="https://colab.research.google.com/github/anhle/AI-Healthcare/blob/master/AI_2D/Ex/Ex_8_Otsu_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Differentiate Between Models** 

Machine learning algorithms aren't always complex, and they can have use cases at many different stages of medical imaging analysis, starting at pre-processing. In this exercise, you'll implement a simple algorithm called Otsu's algorithm that just implements image intensity thresholding to segment breast tissue from the background of an image.

Next, you'll use these masks to extract the image intensity distributions for two types of breasts: fatty and dense. These are standard classifications of breast tissue that radiologists make for all mammography studies. Using image intensity to classify breast tissue type is yet another implementation of Otsu's algorithm.

In [0]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats

import skimage
from skimage import io
import glob

import sklearn
from scipy.ndimage import gaussian_filter

## First we'll do background segmentation:

In [0]:
!git clone https://github.com/anhle/AI-Healthcare.git

In [0]:
import os
dir = '/content/AI-Healthcare/AI_2D/Ex/data'
## Read in two mammo images: 
dense = io.imread(os.path.join(dir,'dense/mdb003.pgm'))
fatty = io.imread(os.path.join(dir,'fatty/mdb005.pgm'))

In [0]:
plt.imshow(dense)

In [0]:
plt.imshow(fatty)

In [0]:
x = plt.hist(dense.ravel(),bins=256)

In [0]:
x = plt.hist(fatty.ravel(),bins=256)

First, I used Otsu’s method to extract background pixels from all of my images using a threshold intensity value of 50. This allowed me to look at the intensity distributions for breast tissue ONLY in fatty tissues or ONLY in dense tissue. I then used scipy.stats.mode to identify the mode (peak) of each type of tissue’s intensity distribution.

Here, fatty tissue had a peak at 140 and dense tissue had a peak at 176.

I then looped through all of the fatty/dense images and again using Otsu’s method with an intensity threshold of 50 to extract background. I then calculated how far each image’s peak was from the peaks of the fatty and dense tissue distributions. Finally, which difference is smaller determines what type of tissue my image most likely is.

In [0]:
## Next, experiment with different cut-off intensity thresholds to try to separate the background of the image
## From the histograms above, we might want to start with zero: 

thresh = 50

dense_bin = (dense > thresh) * 255
fatty_bin = (fatty > thresh) * 255

In [0]:
dense_bin

In [0]:
plt.imshow(dense_bin)

In [0]:
plt.imshow(fatty_bin)

Experiment with different values of 'thresh' above until you are satisfied that you are able to create a reasonable separation of tissue from background.

One image pre-processing trick you might try before binarizing is _smoothing_ which you perform with a gaussian filter. Try adding the following step before binarization: 

img_smooth = gaussian_filter(img, sigma = 5)

Where changing the value of _sigma_ will change the amount of smoothing. 

## Once you have chosen your value of threshold, let's use it to see if we can classify dense v. fatty breast tissue: 

In [0]:
dense_smooth = gaussian_filter(dense, sigma = 3)
plt.imshow(dense_smooth)

In [0]:
## Let's first get all of the intensity values of the breast tissue for our fatty breast images using the
## segmentation method above: 
thresh = 50

fatty_imgs = glob.glob(os.path.join(dir,"fatty/*"))
dense_imgs = glob.glob(os.path.join(dir,"dense/*"))

In [0]:
fatty_intensities = []

for i in fatty_imgs: 
    
    img = plt.imread(i)
    img_mask = (img > thresh)
    fatty_intensities.extend(img[img_mask].tolist())

In [0]:
fatty_intensities

In [0]:
x = plt.hist(fatty_intensities,bins=256)

In [0]:
dense_intensities = []

for i in dense_imgs: 
    
    img = plt.imread(i)
    img_mask = (img > thresh)
    dense_intensities.extend(img[img_mask].tolist())

In [0]:
x = plt.hist(dense_intensities,bins=256)

In [0]:
scipy.stats.mode(fatty_intensities)[0][0]

In [0]:
scipy.stats.mode(dense_intensities)[0][0]

In [0]:
for i in fatty_imgs: 
    
    img = plt.imread(i)
    img_mask = (img > thresh)
    
    fatty_delta = scipy.stats.mode(img[img_mask])[0][0] - scipy.stats.mode(fatty_intensities)[0][0]
    dense_delta = scipy.stats.mode(img[img_mask])[0][0] - scipy.stats.mode(dense_intensities)[0][0]
    
    if (np.abs(fatty_delta) < np.abs(dense_delta)):
        print("Fatty")
    else:
        print("Dense")

In [0]:
for i in dense_imgs: 
    
    img = plt.imread(i)
    img_mask = (img > thresh)
    
    fatty_delta = scipy.stats.mode(img[img_mask])[0][0] - scipy.stats.mode(fatty_intensities)[0][0]
    dense_delta = scipy.stats.mode(img[img_mask])[0][0] - scipy.stats.mode(dense_intensities)[0][0]
    
    if (np.abs(fatty_delta) < np.abs(dense_delta)):
        print("Fatty")
    else:
        print("Dense")

Just by using this method, we're able to get 100% accuracy on dense images and 70% accuracy on fatty images! Now, we 'trained' on the same set of images that we validated on, so in order to be confident in this algorithm we need to validated on a separate held out set, which we'll learn about in the next lesson. 